In [7]:
import mlbstatsapi
import pandas as pd
import numpy as np
import pybaseball

In [32]:
from pybaseball import pitching_stats
pitcher_data = pitching_stats(2021,2023, qual=5)

In [30]:
pitcher_data[pitcher_data['Name'] == 'Lucas Giolito']

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
409,15474,2021,Lucas Giolito,CHW,26,11,9,4.1,3.53,31,...,NaN,NaN,NaN,NaN,104,103,106,NaN,NaN,NaN
833,15474,2022,Lucas Giolito,CHW,27,11,9,1.8,4.90,30,...,NaN,NaN,NaN,NaN,91,100,98,NaN,NaN,NaN
829,15474,2023,Lucas Giolito,- - -,28,8,15,1.0,4.88,33,...,NaN,NaN,NaN,NaN,95,101,101,NaN,NaN,NaN


In [31]:
pitcher_data.pivot_table(index=['Name', 'IDfg'], columns='Season', values='WAR').reset_index()


Season,Name,IDfg,2021,2022,2023
0,A.J. Minter,18655,1.3,2.1,1.4
1,A.J. Puk,19343,NaN,0.3,0.9
2,Aaron Ashby,23550,NaN,0.9,NaN
3,Aaron Bummer,16258,1.2,NaN,0.8
4,Aaron Civale,19479,0.7,1.2,2.5
...,...,...,...,...,...
573,Zach Thompson,16094,1.3,0.2,NaN
574,Zack Greinke,1943,1.2,1.9,1.1
575,Zack Littell,15823,0.3,NaN,1.1
576,Zack Thompson,25918,NaN,NaN,1.1


In [27]:
pitcher_data['IP']

7      167.0
16     213.1
47     205.0
23     178.0
0      175.0
       ...  
125    183.2
121    180.0
126    171.2
127    177.2
94     183.0
Name: IP, Length: 128, dtype: float64

In [21]:
pitcher_data[['IDfg', 'Season', 'Name', 'WAR']]

,IDfg,Season,Name,WAR
7,19361,2021,Corbin Burnes,7.5
16,10310,2021,Zack Wheeler,7.2
47,16149,2022,Aaron Nola,6.3
23,16137,2022,Carlos Rodon,6.2
0,8700,2022,Justin Verlander,6.0
...,...,...,...,...
125,2520,2023,Lance Lynn,0.5
121,7593,2021,Jordan Lyles,0.3
126,9323,2021,Patrick Corbin,0.2
127,7593,2023,Jordan Lyles,0.2


In [24]:
pivoted_data = pitcher_data.pivot_table(index=[  'Name'], columns='Season', values='WAR').reset_index()


In [25]:
pivoted_data

Season,Name,2021,2022,2023
0,Aaron Nola,4.4,6.3,3.9
1,Adam Wainwright,3.9,2.9,NaN
2,Alek Manoah,NaN,4.1,NaN
3,Anthony DeSclafani,3.0,NaN,NaN
4,Blake Snell,NaN,NaN,4.1
...,...,...,...,...
80,Yusei Kikuchi,NaN,NaN,2.6
81,Zac Gallen,NaN,4.2,5.2
82,Zach Eflin,NaN,NaN,4.8
83,Zack Greinke,1.2,NaN,NaN


In [20]:
from pybaseball import batting_stats
batter_data = batting_stats(2021,2023)